In [13]:
import base64
import random
import tensorflow as tf
print(tf.__version__)

2.0.0


In [14]:
## Xs'
M = 5000
N = 6 # 6 Bytes
def data_gen(m = M , n = N) :
    X_int   = [ random.getrandbits(n*8) for _ in range(m) ]
    X_bits  = [ [ 1 if num & (1 << pow2) else 0 for pow2 in range(n*8) ] for num in X_int ]
    X_bytes = [ num.to_bytes(n, 'little') for num in X_int ]
    X_byte_array =  [ [ byte for byte in bytestr ] for bytestr in X_bytes ]
    return X_bits, X_bytes, X_byte_array

random.seed(0)
X, X_bytes, X_B_array = data_gen()
print(X[:2])
print(X_bytes[:2])
print(X_B_array[:2])


[[1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0], [0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]]
[b'\xcd\x07,\xd8\x9fb', b'\xacL\t\xc2\xe7\xe3']
[[205, 7, 44, 216, 159, 98], [172, 76, 9, 194, 231, 227]]


In [15]:
## Ys'
Y_bytes = [ base64.b64encode(s) for s in X_bytes ]
Y_int   = [ int.from_bytes(s, 'little')   for s in Y_bytes ]
Y       = [ [1 if num & 1 << pow2 else 0 for pow2 in range(8*8) ] for num in Y_int ]
Y_B_array = [ [byte for byte in bytestr ] for bytestr in Y_bytes ]

print(Y_bytes[:2])
print(Y[:2])
print(base64.b64decode(Y_bytes[0]))

[b'zQcs2J9i', b'rEwJwufj']
[[0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0], [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0]]
b'\xcd\x07,\xd8\x9fb'


In [16]:
M_train, M_test  = int(M*0.8), int(M*0.2)
assert M_train + M_test == M
# we are trying to reverse it
X_train  = tf.convert_to_tensor(Y_B_array[:M_train], dtype=float)
X_test   = tf.convert_to_tensor(Y_B_array[M_train:], dtype=float)
Y_train  = tf.convert_to_tensor(X_B_array[:M_train], dtype=float)
Y_test   = tf.convert_to_tensor(X_B_array[M_train:], dtype=float)
print("Data Samples :")
print(X_train[0])
print(Y_train[0])
print("X_train: ", X_train.shape)
print("Y_train: ", Y_train.shape)

Data Samples :
tf.Tensor([122.  81.  99. 115.  50.  74.  57. 105.], shape=(8,), dtype=float32)
tf.Tensor([205.   7.  44. 216. 159.  98.], shape=(6,), dtype=float32)
X_train:  (4000, 8)
Y_train:  (4000, 6)


In [17]:
ACTIVATION = 'relu'
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=[8,]),
    tf.keras.layers.Dense(128, activation=ACTIVATION),
    tf.keras.layers.Dense(128, activation=ACTIVATION),
    tf.keras.layers.Dense(128, activation=ACTIVATION),
    tf.keras.layers.Dense(6, activation=ACTIVATION)
])
model.compile(optimizer='adam', loss='mse')

model.summary()

model.fit(X_train, Y_train, epochs=20000)
print("Done " * 100)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_11 (Dense)             (None, 128)               1152      
_________________________________________________________________
dense_12 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_13 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 774       
Total params: 35,022
Trainable params: 35,022
Non-trainable params: 0
_________________________________________________________________
Train on 4000 samples
Epoch 1/20000
4000/4000 [==============================] - 0s 88us/sample - loss: 8640.8763
Ep

4000/4000 [==============================] - 0s 23us/sample - loss: 3712.9735
Epoch 79/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 3709.5058
Epoch 80/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 3698.6525
Epoch 81/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 3679.1462
Epoch 82/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 3686.5069
Epoch 83/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 3693.0141
Epoch 84/20000
4000/4000 [==============================] - 0s 24us/sample - loss: 3682.2463
Epoch 85/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 3668.0127
Epoch 86/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 3643.4106
Epoch 87/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 3655.6603
Epoch 88/20000
4000/4000 [==============================] - 0s 24us/sample - loss: 36

4000/4000 [==============================] - 0s 23us/sample - loss: 2945.2790
Epoch 166/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 2937.0563
Epoch 167/20000
4000/4000 [==============================] - 0s 24us/sample - loss: 2941.8703
Epoch 168/20000
4000/4000 [==============================] - 0s 25us/sample - loss: 2921.4605
Epoch 169/20000
4000/4000 [==============================] - 0s 24us/sample - loss: 2897.5086
Epoch 170/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 2918.9370
Epoch 171/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 2910.0761
Epoch 172/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 2902.2693
Epoch 173/20000
4000/4000 [==============================] - 0s 24us/sample - loss: 2888.8332
Epoch 174/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 2863.0910
Epoch 175/20000
4000/4000 [==============================] - 0s 23us/sample 

4000/4000 [==============================] - 0s 23us/sample - loss: 2349.2634
Epoch 253/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 2329.0456
Epoch 254/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 2336.4101
Epoch 255/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 2321.8084
Epoch 256/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 2331.7284
Epoch 257/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 2303.8733
Epoch 258/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 2298.6715
Epoch 259/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 2284.1500
Epoch 260/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 2305.2448
Epoch 261/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 2289.0818
Epoch 262/20000
4000/4000 [==============================] - 0s 22us/sample 

4000/4000 [==============================] - 0s 25us/sample - loss: 1906.0409
Epoch 340/20000
4000/4000 [==============================] - 0s 25us/sample - loss: 1931.2165
Epoch 341/20000
4000/4000 [==============================] - 0s 24us/sample - loss: 1920.5135
Epoch 342/20000
4000/4000 [==============================] - 0s 25us/sample - loss: 1907.1415
Epoch 343/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 1904.2798
Epoch 344/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 1911.2394
Epoch 345/20000
4000/4000 [==============================] - 0s 25us/sample - loss: 1914.0597
Epoch 346/20000
4000/4000 [==============================] - 0s 25us/sample - loss: 1899.5845
Epoch 347/20000
4000/4000 [==============================] - 0s 24us/sample - loss: 1885.7116
Epoch 348/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 1894.7183
Epoch 349/20000
4000/4000 [==============================] - 0s 23us/sample 

...

4000/4000 [==============================] - 0s 22us/sample - loss: 147.0302
Epoch 19630/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 166.8833
Epoch 19631/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 185.2603
Epoch 19632/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 172.2735
Epoch 19633/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 188.2162
Epoch 19634/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 165.1659
Epoch 19635/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 142.9793
Epoch 19636/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 144.4426
Epoch 19637/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 151.8780
Epoch 19638/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 152.0821
Epoch 19639/20000
4000/4000 [==============================] - 0s 23

4000/4000 [==============================] - 0s 23us/sample - loss: 126.4359
Epoch 19716/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 126.1340
Epoch 19717/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 140.5133
Epoch 19718/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 135.0346
Epoch 19719/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 134.4739
Epoch 19720/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 152.1462
Epoch 19721/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 174.3615
Epoch 19722/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 171.3469
Epoch 19723/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 156.1877
Epoch 19724/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 142.4182
Epoch 19725/20000
4000/4000 [==============================] - 0s 23

4000/4000 [==============================] - 0s 23us/sample - loss: 153.2744
Epoch 19802/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 152.3111
Epoch 19803/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 188.6879
Epoch 19804/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 267.2779
Epoch 19805/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 162.1704
Epoch 19806/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 140.2240
Epoch 19807/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 126.2091
Epoch 19808/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 117.6290
Epoch 19809/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 127.5781
Epoch 19810/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 139.8891
Epoch 19811/20000
4000/4000 [==============================] - 0s 22

4000/4000 [==============================] - 0s 23us/sample - loss: 166.6323
Epoch 19888/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 157.6599
Epoch 19889/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 175.7209
Epoch 19890/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 150.1857
Epoch 19891/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 139.5349
Epoch 19892/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 131.2903
Epoch 19893/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 162.0085
Epoch 19894/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 158.5906
Epoch 19895/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 143.3443
Epoch 19896/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 142.3810
Epoch 19897/20000
4000/4000 [==============================] - 0s 23

4000/4000 [==============================] - 0s 24us/sample - loss: 147.9189
Epoch 19974/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 184.9786
Epoch 19975/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 167.3723
Epoch 19976/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 175.1112
Epoch 19977/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 169.0400
Epoch 19978/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 151.4957
Epoch 19979/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 136.4132
Epoch 19980/20000
4000/4000 [==============================] - 0s 23us/sample - loss: 151.6772
Epoch 19981/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 145.5775
Epoch 19982/20000
4000/4000 [==============================] - 0s 22us/sample - loss: 174.1868
Epoch 19983/20000
4000/4000 [==============================] - 0s 23

In [19]:
#print(predictions[:100])
model.evaluate(X_test, Y_test)
# Y_hat = model.predict(X_test)

# print(Y_test[0])
# correct = sum([ int(''.join([str(b) for b in x]),2) == int(''.join([str(b) for b in y]), 2) for x,y in zip(predictions, Y_test)])
#correct  = sum([ y_hat == y for y_hat, y in zip(predictions, Y_test)])
#accuracy = correct / float(M_test)
#print("Correct cases : {}/{}".format(correct, M_test) )
#print("accuracy      : {}%".format(accuracy* 100.0 ) )

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

10446.2428125